# Kerasを用いた画像分類入門

## ハンズオン テンプレート

## Google Colab用の設定

Google Colaboratory で実行する場合には下記のセルを実行してください。

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/My\ Drive/GD_share01/noodles.zip -d /content

## ローカルPC用の設定

各自のPCでJupyter Notebookを起動して実行する場合には、以下のセルを実行してください。
また、別途noodles.zipを解凍しておいてください。

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install tensorflow
!pip install keras
!pip install pillow

## ライブラリのインポート

In [ ]:
import keras
## import tensorflow.keras as keras  #　こちらでもOK

## 画像データの準備

下記のプログラムで、テスト用データのジェネレータのプログラムを完成させてください。

In [ ]:
train_dir = './noodles/train/'
validation_dir = './noodles/validaiton/'
test_dir = './noodles/test/'
image_size = (128, 128)
batch_size = 32
steps_per_epoch = 300 // batch_size
validation_steps = 100 // batch_size
test_steps = 100 // batch_size

image_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

train_generator = image_data_generator.flow_from_directory(
    train_dir,
    target_size=image_size, 
    batch_size=batch_size
)
validation_generator = image_data_generator.flow_from_directory(
    validation_dir,
    target_size=image_size, 
    batch_size=batch_size
)

### 問題1.

以下のtest_generatorを、完成させなさい。

In [ ]:
test_generator = 

## モデルの構築

全結合層「Dense」を使って、多層のニューラルネットワークを完成させましょう。

### 問題2.

以下の途中にある`model.add()`の引数を書いてください。

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=(128,128,3)))
model.add(keras.layers.Dense(128, activation='relu'))
model.add()    ## このmodel.add()の引数を書いてください
model.add(keras.layers.Dense(2, activation='softmax'))

## モデル構造の表示

In [ ]:
model.summary()

## モデルのコンパイル

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['acc'])

## モデルの訓練

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

## 学習経過のグラフ化

下記のプログラムで、エポックごとの損失と正解率をグラフ化できます。
グラフから何が読み取れるでしょうか？

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend()
plt.show()

## モデルの評価

In [ ]:
evaluation = model.evaluate_generator(test_generator, steps=test_steps)
print(evaluation)   # [loss, accuracy] が表示される

## モデルの保存

In [ ]:
model.save('noodle_model.h5')

## 保存済みモデルの読み込み

In [ ]:
model = keras.models.load_model('noodle_model.h5')

## モデルによる予測

下記のプログラムで、学習したモデルを使ってテスト用データの分類（予測）を行うことができます。
予測結果がどのような形式なのかを確認してください。

In [ ]:
predictions = model.predict_generator(test_generator, steps=test_steps)

print(predictions)

## モデルの予測結果を見てみる

次のプログラムを実行すると、テスト用データの１バッチ分についてモデルの予測値と正解を画像つきで確認できます。
どのような結果になったかを確認してください。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

code_to_class = {0: 'ラーメン', 1: 'うどん'}

x, y = next(test_generator)
y_true = np.argmax(y, axis=1)
y_prob = model.predict_on_batch(x)
y_pred = np.argmax(y_prob, axis=1)

for i in range(len(x)):
    plt.imshow(x[i])
    plt.show()
    print('推定：', code_to_class[y_pred[i]])
    print('正解：', code_to_class[y_true[i]])